In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR 
import time
import DenseModule
import DenseCopied
import sys
sys.executable

'/home/slavko/anaconda3/envs/DeepLearningPyTorch/bin/python'

In [2]:
import struct
print(struct.calcsize("P") * 8)

64


In [3]:
#constants and net definition
learning_rate = 0.1
numEpochs = 300
lr_decay = 0.98
dataAugmentation = False

ALPHA_CHANGING = False
MIN_ALPHA = 1
MAX_ALPHA = 3
alpha = 10**(MIN_ALPHA)

net = DenseModule.GeneralConvNet([4,4,4,4], 16, 16,
                                 convDropRegime = "None", poolDropRegime = "None",
                                 pConvDrop = 0.2, pPoolDrop = [0.9,0,0,0],
                                 bottleDrop = False, convDrop = True, convPoolDrop = True, 
                                 finalDrop = False, inputDependentPool = True, reversedBnOrder = False)
#net = DenseCopied.DenseNet121()
#net = DenseModule.NearestNeighborNet(10, 3)
#net = DenseModule.DenseNormNet(4, 10)
#net = DenseModule.QuadraticNet(2, 20)

In [4]:
#import, load and normalize CIFAR

trainBatchSize = 128
testBatchSize = 128

transform_train = []
if (dataAugmentation):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trainBatchSize,
                                          shuffle=True, num_workers=4)


train_test_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_test)
train_test_loader = torch.utils.data.DataLoader(train_test_set, batch_size=trainBatchSize,
                                          shuffle=True, num_workers=4)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=testBatchSize,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
def testPerformance(dataLoader, dataName):
    correct = 0
    total = 0
    loss = 0
    testCriterion = nn.CrossEntropyLoss(size_average=False)
    with torch.no_grad():
        net.eval()
        for data in dataLoader:
            inputs, labels = data
            if cuda_avail:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            outputs = net(inputs)
            loss += testCriterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Loss and accuracy on the %s set: %.5f, %.2f %%' % (dataName, loss / total, 100 * correct / total))

In [6]:
#Train on training set
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay = 5e-4)

def adjust_learning_rate():
    global learning_rate
    learning_rate *= lr_decay
    for param_group in optimizer.param_groups:
        param_group["lr"] = learning_rate

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
#scheduler = MultiStepLR(optimizer, milestones=[150,250], gamma=0.1)
cuda_avail = torch.cuda.is_available()
if cuda_avail:
    net.cuda()

net.printNet()
print("Number of epochs: ", numEpochs)

flag = 0

for epoch in range(numEpochs):  # loop over the dataset multiple times
    timeStart = time.time()
    #scheduler.step()
    net.train()
    running_loss = 0.0
    numBatches = 0
    if (ALPHA_CHANGING):
        alpha = 10 ** ((epoch / numEpochs) * (MAX_ALPHA - MIN_ALPHA) + MIN_ALPHA)
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        if cuda_avail:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        if (ALPHA_CHANGING):
            outputs = net(inputs, alpha)
        else:
            outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        '''if (flag == 0):
            save('s.dot', loss.grad_fn)
            flag = 1'''
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        numBatches += 1
    timeElapsed = time.time() - timeStart
    adjust_learning_rate()
        
    print('[%d] loss: %.3f LR: %.5f Epoch time: %.2f s, Remaining time: %.2f s alpha: %.2f' %
          (epoch + 1, running_loss / numBatches, learning_rate, timeElapsed, (numEpochs - epoch - 1) * timeElapsed, alpha))
    
    if ((epoch + 1) % 10 == 0):
        testPerformance(train_test_loader, "train")
        testPerformance(testloader, "test")

print('Finished Training')
testPerformance(train_test_loader, "train")
testPerformance(testloader, "test")

GeneralConvNet will be trained!
Number of parameters:  78923
Number of epochs:  300


Process Process-3:
Process Process-1:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-2:
  File "/home/slavko/anaconda3/envs/DeepLearningPyTorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/slavko/anaconda3/envs/DeepLearningPyTorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/slavko/anaconda3/envs/DeepLearningPyTorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/slavko/anaconda3/envs/DeepLearningPyTorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/slavko/anaconda3/envs/DeepLearningPyTorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/slavko/anaconda3/envs/DeepLearning

KeyboardInterrupt: 

In [ ]:
#Distribution over classes in test set

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    net.eval()
    for data in testloader:
        inputs, labels = data
        if cuda_avail:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %.2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
torch.arange(20, dtype=torch.long)